In [94]:
import numpy as np
import math

In [95]:
v_states = [
            [1,1,1,0,0,0,0,0,0],
            [0,0,0,1,1,1,0,0,0],
            [0,0,0,0,0,0,1,1,1],
            [1,0,0,1,0,0,1,0,0],
            [0,1,0,0,1,0,0,1,0],
            [0,0,1,0,0,1,0,0,1],
            [1,0,0,0,1,0,0,0,1],
            [0,0,1,0,1,0,1,0,0],
            ]

def check_victory(s):
    # s is a length 9 array
    x = s == 1
    o = s == 2
    sx = v_states @ x
    so = v_states @ o
    if sx.max() == 3:
        return 1
    elif so.max() == 3:
        return -1
    else:
        return 0

def check_victory_mat(s):
    # s is matrix of n x 9 dimension
    # matrix product with vicory states: s @ v = n x 9 @ 9 x 8 -> n x 8 
    # a value of 3 in the output means a victory 
    x = s == 1
    o = s == 2
    sx = v_states @ np.transpose(x)
    so = v_states @ np.transpose(o)
    sx = sx.max(axis = 0)
    so = so.max(axis = 0)
    sx = (sx == 3) * 1
    so = (so == 3) * -1
    return so + sx
   
s_test = np.array([[2,1,1,0,0,1,2,2,1],
                  [2,1,2,0,0,1,2,2,1],
                  [2,1,2,0,0,2,2,2,2]])
for s in s_test:
    print(check_victory(s))

check_victory_mat(s_test)    

1
0
-1


array([ 1,  0, -1])

In [96]:
def print_board(s):
    x = ['-', 'x', 'o']
    print(f' [{x[s[0]]} {x[s[1]]} {x[s[2]]}] \n [{x[s[3]]} {x[s[4]]} {x[s[5]]}] \n [{x[s[6]]} {x[s[7]]} {x[s[8]]}] \n')

def print_states(states):
    print(len(states)) 
    for s in states:
        print(len(s)) 

# get all tick tack toe states. 0 is empty, 1 is x and 2 is o
board = [0 for i in range(9)]
states = [[board]]
current_round_states = [board]
next_round_states = []
connections = []
current_connections = []
# iteration over all steps to fill the board
for i in range(9):
    #iteration over all states of the previous step
    for j, s in enumerate(current_round_states):
        # check if the step is even. If yes, add a x otherwise an o
        if i % 2 == 0:
            x = 1
        else:
            x = 2    
        # go through all fields
        c = []
        # check if state was victorios. Only calculate next states if the state wasn't victorios 
        if check_victory(np.array(s)) == 0:
            for k in range(9):
                new_state = list(s).copy()
                # check if field is taken
                if s[k] == 0:
                    new_state[k] = x
                    if new_state in next_round_states:
                        c.append(next_round_states.index(new_state))
                    else:
                        next_round_states.append(new_state)
                        c.append(next_round_states.index(new_state))
        current_connections.append(c)
    connections.append(current_connections)
    current_connections = []
    states.append(next_round_states)
    # remove states that won from the next round
    victories = check_victory_mat(np.array(next_round_states))
    current_round_states = np.array(next_round_states)
    next_round_states = []

    
print_states(states)
print('\n')
print_states(connections)

10
1
9
72
252
756
1260
1520
1140
390
78


9
1
9
72
252
756
1260
1520
1140
390


In [97]:
V = []
for s in states[0:]:
    V.append(np.random.rand(len(s)))

print_states(V)


10
1
9
72
252
756
1260
1520
1140
390
78


In [98]:
def calculate_V(turn, i, verbose = False):

    actions = connections[turn][i] 
    V_new = []
    for a in actions:
        s = states[turn + 1][a]
        reward = check_victory(np.array(s))
        if reward == 0 and turn != 8:
            opponent_actions = connections[turn + 1][a]
            V_new.append( sum([check_victory(np.array(states[turn + 2][p])) + gamma * V[turn + 2][p] for p in opponent_actions])/ len(opponent_actions))

        else:
            V_new.append(reward)
    
    if verbose == True:
        print(V_new)
        
    V_max = max(V_new)
    a = np.argmax(V_new)
    return V_max, actions[a]


    

In [99]:
turn = 4
state_index = 0
V_new, a_best = calculate_V(turn,state_index,verbose=True)
print(V[turn][state_index])
print_board(states[turn][state_index])
print(V_new, a_best)
print_board(states[turn + 1][a_best])
for a in connections[turn][state_index]:
    print(a)
    print_board(states[turn + 1][a])

[0.19762091128892073, 0.16092837583370379, 0.103926485503759, 0.3024311973129377, 0.1175160263473463]
0.18687454185162944
 [x o x] 
 [o - -] 
 [- - -] 

0.3024311973129377 3
 [x o x] 
 [o - -] 
 [- x -] 

0
 [x o x] 
 [o x -] 
 [- - -] 

1
 [x o x] 
 [o - x] 
 [- - -] 

2
 [x o x] 
 [o - -] 
 [x - -] 

3
 [x o x] 
 [o - -] 
 [- x -] 

4
 [x o x] 
 [o - -] 
 [- - x] 



In [100]:
V[9]

array([0.44378754, 0.21325945, 0.96774849, 0.72397771, 0.99260755,
       0.78691219, 0.78015763, 0.92135634, 0.93874978, 0.53779491,
       0.94897887, 0.72651936, 0.48316018, 0.07216315, 0.96850441,
       0.51061683, 0.71695811, 0.5974424 , 0.67022812, 0.26761033,
       0.00823849, 0.0873056 , 0.1275261 , 0.71335298, 0.59723463,
       0.2410541 , 0.46893645, 0.43326383, 0.9581921 , 0.72656046,
       0.35641472, 0.50470852, 0.26784399, 0.82707742, 0.74802306,
       0.88590514, 0.43291375, 0.54753439, 0.70006639, 0.45462145,
       0.24184615, 0.9525461 , 0.36753923, 0.3838584 , 0.08251458,
       0.23179691, 0.18999287, 0.02917733, 0.53111236, 0.75324436,
       0.95916397, 0.6236317 , 0.19429033, 0.6377751 , 0.15728648,
       0.88952578, 0.69025852, 0.04129391, 0.43551231, 0.37382844,
       0.00629313, 0.94674829, 0.82689182, 0.23085637, 0.89096345,
       0.313932  , 0.42545063, 0.99487294, 0.49389631, 0.23651833,
       0.81152419, 0.09629422, 0.8819007 , 0.0112919 , 0.67962

In [101]:
def get_next_state(turn, i, a):
    if np.random.random_sample() < epsilon:
        a = np.random.choice(connections[turn][i])    
    if check_victory(np.array(states[turn + 1][a])) != 0:
        return None    
    opponent_actions = connections[turn + 1][a]
    p = np.random.choice(opponent_actions)
    if check_victory(np.array(states[turn + 2][p])) != 0:
        return None   
    return p


In [102]:
choice = np.random.choice(connections[2][3])
print(connections[2][3])
print(choice)

[21, 22, 23, 24, 25, 26, 27]
25


In [103]:
def get_best_action(V, turn, state_index):
    actions = connections[turn][state_index]
    v_arr = [V[turn + 1][a] for a in actions]
    best_action = actions[np.argmax(v_arr)] 
    return best_action
action = get_best_action(V, 5, 5)   


def play_game(V, verbose = False):
    a = 0
    for t in range(9):
        s = states[t][a]
        if verbose:
            print_board(s)
        reward = check_victory(np.array(s))
        if reward != 0:
            return reward    
        elif t%2 != 0:
            next_moves = connections[t][a]
            a = np.random.choice(next_moves)
        else:
            a = get_best_action(V, t, a)  
    return 0


n = 1000
rewards = sum([play_game(V) for i in range(n)])/n
print(rewards)

0.244


In [104]:
gamma = 0.7 # discount for future rewards
epsilon = 0.5 # epsilon determines the exploration of not optimal actions. Higher epsilon = more exploration. between 0 and 1

def choose_next_action(best_action, possible_actions):
    if np.random.random_sample() < epsilon:
        action = np.random.choice(possible_actions)
    else: 
        action = best_action
    return action

def assign_V(V_new, turn, state_index):
    residuum = np.abs(V[turn][state_index] - V_new)
    V[turn][state_index] = V_new
    return residuum

V = []
#for s in states[0:]:
#    V.append(np.random.rand(len(s)))
for s in states[0:]:
    V.append(np.zeros(len(s)))    

for i in range(200):
    residuum = []
    state_index = 0 # innitial starting board
    for turn in np.arange(0,9):
        possible_actions = connections[turn][state_index]
        if turn%2 == 0:
            V_new, best_action = calculate_V(turn, state_index)
            res = assign_V(V_new, turn, state_index)
            residuum.append(res)
            action = choose_next_action(best_action, possible_actions)    
        else: 
            action = np.random.choice(possible_actions)
        #print(turn, state_index, action)
        #print_board(states[turn + 1][action])
        state_index = action
        if check_victory(np.array(states[turn + 1][action])) != 0:
            break  
    #print(i, np.sum(residuum)/len(residuum))
    if i > 0 and i%10 == 0:
        n = 1000
        rewards = sum([play_game(V) for i in range(n)])/n
        print(rewards)

0.54
0.528
0.48
0.473
0.525
0.502
0.497
0.437
0.501
0.513
0.511
0.498
0.536
0.451
0.518
0.472
0.454
0.505
0.43


In [105]:
play_game(V, verbose=True)

 [- - -] 
 [- - -] 
 [- - -] 

 [x - -] 
 [- - -] 
 [- - -] 

 [x - -] 
 [- - -] 
 [- - o] 

 [x x -] 
 [- - -] 
 [- - o] 

 [x x -] 
 [- - -] 
 [o - o] 

 [x x x] 
 [- - -] 
 [o - o] 



1

In [106]:
gamma = 0.5 # discount for future rewards
epsilon = 0.0 # epsilon determines the exploration of not optimal actions. Higher epsilon = more exploration. between 0 and 1

V = []
for s in states[0:]:
    V.append(np.random.rand(len(s)))

state_index = 0 # innitial starting board
action = 0
for turn in np.arange(0,9,2):
    V_new, action = calculate_V(turn, state_index)
    V[turn][state_index] = V_new
    print(turn, state_index, action)
    print_board(states[turn][state_index])
    if turn < 8:
        state_index = get_next_state(turn, state_index, action)
        if not state_index:
            break    
    


0 0 5
 [- - -] 
 [- - -] 
 [- - -] 

2 43 221
 [- - -] 
 [o - x] 
 [- - -] 

4 682 886
 [- - -] 
 [o - x] 
 [o - x] 



In [107]:
a = 0
for t in range(9):
    s = states[t][a]
    print_board(s)
    next_moves = connections[t][a]
    if not next_moves:
        break
    a = np.random.choice(next_moves)

 [- - -] 
 [- - -] 
 [- - -] 

 [- - -] 
 [- x -] 
 [- - -] 

 [- - -] 
 [- x -] 
 [- o -] 

 [- - x] 
 [- x -] 
 [- o -] 

 [- - x] 
 [- x o] 
 [- o -] 

 [- - x] 
 [- x o] 
 [x o -] 

